In [81]:
!pip -qq install backtesting
!pip -qq install ta

In [82]:
import pandas as pd
import datetime as dt

df = pd.DataFrame(pd.read_excel('/content/BankNiftyFutures_Data5Min.xlsx'))

In [83]:
backtest_df = df.copy()
backtest_df.loc[:,'Datetime'] = pd.to_datetime(df.Date.astype(str)+' '+df.Time.astype(str), errors='coerce')
backtest_df.set_index('Datetime', append=False, inplace=True)
backtest_df = backtest_df[['Open 5',	'High 5',	'Low 5',	'Close 5',	'Volume 5']]
backtest_df.columns = ['Open',	'High',	'Low',	'Close',	'Volume']

# Backtesting using python

In [84]:
# Backtest a simple EMA -> SMA crossover strategy on above data

import backtesting
backtesting.set_bokeh_output(notebook=True)
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

import ta.trend as tr


class EmaSmaCross(Strategy):
    def init(self):
        cp = pd.Series(self.data.Close)
        
        self.ema1 = self.I(tr.ema_indicator,close=cp, window=200);
        self.ema2 = self.I(tr.sma_indicator,close=cp, window=2000)
        

    def next(self):
        # Buy or sell on ema -> sma cross over
        # Have a 200 pts stop loss for long trades and 130 points for short trades
        if crossover(self.ema1, self.ema2):
            self.buy(sl=self.data.High-200)
        elif crossover(self.ema2, self.ema1):
            self.sell(sl=self.data.Low+130)
            
bt = Backtest(backtest_df, EmaSmaCross, commission=.002,cash=1000000,
              exclusive_orders=True)
# Run the backtest
stats = bt.run()
 
# Print the performance metrics for this strategy 
print(stats)

# Plot the price movements, trades and other meta-data such as drawdown etc.
bt.plot(resample=True);

Start                     2015-01-01 09:20:00
End                       2019-12-31 15:26:00
Duration                   1825 days 06:06:00
Exposure Time [%]                     55.9284
Equity Final [$]                  1.49445e+06
Equity Peak [$]                   1.67499e+06
Return [%]                            49.4454
Buy & Hold Return [%]                 72.1571
Return (Ann.) [%]                     8.60862
Volatility (Ann.) [%]                 12.5463
Sharpe Ratio                         0.686147
Sortino Ratio                         1.10342
Calmar Ratio                         0.433907
Max. Drawdown [%]                    -19.8398
Avg. Drawdown [%]                   -0.523837
Max. Drawdown Duration      419 days 05:50:00
Avg. Drawdown Duration        5 days 04:10:00
# Trades                                   69
Win Rate [%]                          20.2899
Best Trade [%]                        26.4766
Worst Trade [%]                      -2.00401
Avg. Trade [%]                    

/usr/local/lib/python3.7/dist-packages/backtesting/_plotting.py:104: UserWarning: Data contains too many candlesticks to plot; downsampling to '8H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: Superimposed OHLC plot matches the original plot. Skipping.


# To 15 mins time-frame

In [85]:
df = pd.DataFrame(pd.read_excel('/content/BankNiftyFutures_Data5Min.xlsx'))

df.loc[:,'Datetime'] = pd.to_datetime(df.Date.astype(str)+' '+df.Time.astype(str), errors='coerce')
df.set_index('Datetime', append=False, inplace=True)

conv = {                                                                                                             
          'Open 5':'first',                                                                                                    
          'High 5':'max',                                                                                                       
          'Low 5':'min',                                                                                                        
          'Close 5': 'last',                                                                                                    
          'Volume 5': 'last',
          'Year': 'last',
       }

df = df.resample('15min').agg(conv)
df.dropna(inplace = True)

df.columns = ['Open 15',	'High 15',	'Low 15',	'Close 15',	'Volume 15', 'Year']
df.to_csv('15mins.csv')
df.head()

,Open 15,High 15,Low 15,Close 15,Volume 15,Year
Datetime,,,,,,
2015-01-01 09:15:00,18861.3496,18867.9492,18793.0,18829.0000,40925.0,2015.0
2015-01-01 09:30:00,18826.9492,18833.4004,18775.0,18803.9492,60100.0,2015.0
2015-01-01 09:45:00,18803.9492,18850.0000,18800.0,18846.8008,30750.0,2015.0
2015-01-01 10:00:00,18846.9004,18858.5996,18830.0,18850.0000,13800.0,2015.0
2015-01-01 10:15:00,18848.5508,18858.0000,18830.0,18840.2500,4250.0,2015.0


In [86]:
df = pd.read_csv('/content/15mins.csv')

df.loc[:,'dt'] = pd.to_datetime(df['Datetime'], errors='coerce')
df['dt'] = df['dt'].dt.strftime('%Y-%m-%d')

# VWAP

In [87]:
def calculateVwap(data):
    data['TP'] = (data['High 15']+data['Low 15']+data['Close 15'])/3.0
    data['TradedValue']  = data['TP']*data['Volume 15']
    data['CumVolume'] = data['Volume 15'].cumsum()
    data['CumTradedValue'] = data['TradedValue'].cumsum()
    data['VWAP'] = data['CumTradedValue'] /data['CumVolume']
    return data

df = df.groupby('dt').apply(calculateVwap)

df.head()

,Datetime,Open 15,High 15,Low 15,Close 15,Volume 15,Year,dt,TP,TradedValue,CumVolume,CumTradedValue,VWAP
0,2015-01-01 09:15:00,18861.3496,18867.9492,18793.0,18829.0000,40925.0,2015.0,2015-01-01,18829.983067,7.706171e+08,40925.0,7.706171e+08,18829.983067
1,2015-01-01 09:30:00,18826.9492,18833.4004,18775.0,18803.9492,60100.0,2015.0,2015-01-01,18804.116533,1.130127e+09,101025.0,1.900744e+09,18814.595008
2,2015-01-01 09:45:00,18803.9492,18850.0000,18800.0,18846.8008,30750.0,2015.0,2015-01-01,18832.266933,5.790922e+08,131775.0,2.479837e+09,18818.718792
3,2015-01-01 10:00:00,18846.9004,18858.5996,18830.0,18850.0000,13800.0,2015.0,2015-01-01,18846.199867,2.600776e+08,145575.0,2.739914e+09,18821.323902
4,2015-01-01 10:15:00,18848.5508,18858.0000,18830.0,18840.2500,4250.0,2015.0,2015-01-01,18842.750000,8.008169e+07,149825.0,2.819996e+09,18821.931684


In [88]:
closes = df['Close 15'].tolist()
VWAPS = df['VWAP'].tolist()

action = []
for i in range(len(closes)):
    if closes[i] > VWAPS[i]:
        action.append(str('Buy'))
    else:
        action.append(str('Sell'))

df['action'] = action

In [89]:
'''
A tick is a measure of the minimum upward or downward movement in the price of a security

Therefore we'll use 1 tick = (high - low)/2'''

"\nA tick is a measure of the minimum upward or downward movement in the price of a security\n\nTherefore we'll use 1 tick = (high - low)/2"

In [90]:
df['1tick'] = abs(df['Close 15'] - df['Open 15']) / 2
df['SL'] = df['Low 15'] - df['1tick']

df.head()

,Datetime,Open 15,High 15,Low 15,Close 15,Volume 15,Year,dt,TP,TradedValue,CumVolume,CumTradedValue,VWAP,action,1tick,SL
0,2015-01-01 09:15:00,18861.3496,18867.9492,18793.0,18829.0000,40925.0,2015.0,2015-01-01,18829.983067,7.706171e+08,40925.0,7.706171e+08,18829.983067,Sell,16.1748,18776.8252
1,2015-01-01 09:30:00,18826.9492,18833.4004,18775.0,18803.9492,60100.0,2015.0,2015-01-01,18804.116533,1.130127e+09,101025.0,1.900744e+09,18814.595008,Sell,11.5000,18763.5000
2,2015-01-01 09:45:00,18803.9492,18850.0000,18800.0,18846.8008,30750.0,2015.0,2015-01-01,18832.266933,5.790922e+08,131775.0,2.479837e+09,18818.718792,Buy,21.4258,18778.5742
3,2015-01-01 10:00:00,18846.9004,18858.5996,18830.0,18850.0000,13800.0,2015.0,2015-01-01,18846.199867,2.600776e+08,145575.0,2.739914e+09,18821.323902,Buy,1.5498,18828.4502
4,2015-01-01 10:15:00,18848.5508,18858.0000,18830.0,18840.2500,4250.0,2015.0,2015-01-01,18842.750000,8.008169e+07,149825.0,2.819996e+09,18821.931684,Buy,4.1504,18825.8496


In [91]:
new_df = df[df.action == "Sell"]
new_df['profit_or_loss'] = new_df['Close 15'] - new_df['Open 15']
merged = pd.concat([new_df.profit_or_loss[new_df.profit_or_loss > 0], new_df.profit_or_loss[new_df.profit_or_loss < 0]], axis = 1)
merged.columns = ['profit', 'loss']
merged.fillna(0, inplace = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [92]:
profit_loss_timestamps = pd.concat([new_df[['Datetime', 'Year']], merged], axis = 1)

profit_loss_timestamps.loc[:,'date'] = pd.to_datetime(profit_loss_timestamps['Datetime'], errors='coerce')
profit_loss_timestamps['date'] = profit_loss_timestamps['date'].dt.strftime('%Y-%m-%d')
profit_loss_timestamps

,Datetime,Year,profit,loss,date
0,2015-01-01 09:15:00,2015.0,0.0000,-32.3496,2015-01-01
1,2015-01-01 09:30:00,2015.0,0.0000,-23.0000,2015-01-01
50,2015-01-05 09:15:00,2015.0,0.0000,-55.0000,2015-01-05
51,2015-01-05 09:30:00,2015.0,0.4492,0.0000,2015-01-05
52,2015-01-05 09:45:00,2015.0,4.0996,0.0000,2015-01-05
...,...,...,...,...,...
30635,2019-12-31 14:15:00,2019.0,0.0000,-34.1504,2019-12-31
30636,2019-12-31 14:30:00,2019.0,16.2500,0.0000,2019-12-31
30637,2019-12-31 14:45:00,2019.0,0.0000,-43.1504,2019-12-31
30638,2019-12-31 15:00:00,2019.0,1.3496,0.0000,2019-12-31


# Day-wise profit

In [93]:
profit_loss_timestamps.groupby('date')['profit'].sum()

date
2015-01-01      0.0000
2015-01-05     81.6484
2015-01-06    193.5000
2015-01-07    169.0508
2015-01-08     16.0488
                ...   
2019-12-23     23.4512
2019-12-24     21.5996
2019-12-26     84.3964
2019-12-30    190.9512
2019-12-31     39.6484
Name: profit, Length: 1147, dtype: float64

#Day-wise loss

In [94]:
profit_loss_timestamps.groupby('date')['loss'].sum()

date
2015-01-01    -55.3496
2015-01-05   -230.4512
2015-01-06   -841.9572
2015-01-07   -359.2500
2015-01-08   -117.5528
                ...   
2019-12-23   -177.0000
2019-12-24   -132.6504
2019-12-26   -389.9492
2019-12-30   -356.7520
2019-12-31   -315.8516
Name: loss, Length: 1147, dtype: float64

#Year-wise profit

In [95]:
profit_loss_timestamps.groupby('Year')['profit'].sum() # 2019 was the best 

Year
2015.0    21112.4382
2016.0    17681.7918
2017.0    14483.5168
2018.0    17787.4440
2019.0    26153.6592
Name: profit, dtype: float64

# Year-wise loss

In [96]:
profit_loss_timestamps.groupby('Year')['loss'].sum()

Year
2015.0   -74292.8018
2016.0   -60240.3606
2017.0   -43629.7052
2018.0   -66149.0464
2019.0   -88895.2896
Name: loss, dtype: float64

# aggregate profit/loss

In [97]:
profit_loss_timestamps.groupby('Year')['profit'].sum() + profit_loss_timestamps.groupby('Year')['loss'].sum()

Year
2015.0   -53180.3636
2016.0   -42558.5688
2017.0   -29146.1884
2018.0   -48361.6024
2019.0   -62741.6304
dtype: float64

# It is estimated that 2017 was the best performing year